Following the blog: https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718

In [26]:
from __future__ import unicode_literals, print_function
import spacy
import pandas as pd
import json
import logging
import sys

import pickle
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
path ='/Users/kinga/Documents/4_th_Brain/glg_sandbox/'
input_file = path  + "Data/ner_dataset.json"
output_file = path + "Data/dump1.p"

Downloaded the 'ner_dataset.csv' dataset from kaggle: https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus?select=ner_dataset.csv part of which will be used to train the spacy model

In [10]:
docLoc = "/Users/kinga/Documents/4_th_Brain/glg_sandbox/Data/ner_dataset.csv"
df = pd.read_csv(docLoc, encoding = "ISO-8859-1", error_bad_lines=False)

In [11]:
df.shape


(1048575, 4)

In [4]:
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [21]:
df = df.drop(columns = ['Sentence #', "POS"])

In [23]:
df.to_csv('Data/ner_dataset.tsv', sep='\t', encoding='utf-8', index=False) #converting the .csv to .tsv file

In [13]:
# Convert .tsv file to dataturks json format. 
import json
import logging
import sys
def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None



In [15]:
tsv_to_json_format("Data/ner_dataset.tsv",'Data/ner_dataset.json','abc') #converting .tsv to .json file

In [16]:
path ='/Users/kinga/Documents/4_th_Brain/glg_sandbox/'
input_file = path  + "Data/ner_dataset.json"
output_file = path + "Data/dump1.p"
try:
        training_data = []
        lines=[]
        with open(input_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

#         print(training_data)

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))

In [32]:
type(training_data)

list

In [33]:
training_data[0]

('Word Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country ',
 {'entities': [(0, 4, 'Tag'),
   (53, 59, 'B-geo'),
   (82, 86, 'B-geo'),
   (116, 123, 'B-gpe')]})

In [34]:
len(training_data) 

47761

In [82]:
# Loading training data 
with open (output_file, 'rb') as fp:
    training_data = pickle.load(fp)

In [91]:
# sentence_count = 260 
TRAIN_DATA = training_data[:260] 

In [92]:
# Training additional entity types using spaCy
from __future__ import unicode_literals, print_function
import pickle
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

In [93]:
# New entity labels
# Specify the new entity labels which you want to add here
LABEL = ['I-geo', 'B-geo', 'I-art', 'B-art', 'B-tim', 'B-nat', 'B-eve', 'O', 'I-per', 'I-tim', 'I-nat', 'I-eve', 'B-per', 'I-org', 'B-gpe', 'B-org', 'I-gpe']

"""
geo = Geographical Entity
org = Organization
per = Person
gpe = Geopolitical Entity
tim = Time indicator
art = Artifact
eve = Event
nat = Natural Phenomenon
"""

'\ngeo = Geographical Entity\norg = Organization\nper = Person\ngpe = Geopolitical Entity\ntim = Time indicator\nart = Artifact\neve = Event\nnat = Natural Phenomenon\n'

In [94]:
nlp = spacy.blank('en')  # create blank Language class
print("Created blank 'en' model")

Created blank 'en' model


In [95]:

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

In [96]:
#adding the labels
for i in LABEL:
    ner.add_label(i)

In [97]:
optimizer = nlp.begin_training()

In [98]:
n_iter = 1000
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35,
                           losses=losses)
        if itn%10 == 0:
            print(f'Losses for iteration {itn} is {losses}.')

KeyError: "[E022] Could not find a transition with the name 'U-Tag' in the NER model."

In [55]:
# Test the trained model
test_text = 'Gianni Infantino is the president of FIFA.'
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

Entities in 'Gianni Infantino is the president of FIFA.'
B-per Gianni
B-geo Infantino
B-org FIFA
B-tim .


In [54]:
# output_dir = "/Users/kinga/Documents/4_th_Brain/glg_sandbox/Data/"

# nlp.to_disk("model1")
# print("Saved model to", output_dir)

Saved model to Data/


In [7]:
# Test the saved model
test_text = 'Gianni Infantino is the president of FIFA.'
# print("Loading from", output_dir)
nlp2 = spacy.load('/Users/kinga/Documents/4_th_Brain/glg_sandbox/model1')
doc2 = nlp2(test_text)
for ent in doc2.ents:
    print(ent.label_, ent.text)

B-per Gianni
B-geo Infantino
B-org FIFA
B-tim .


Trying model on GLG Case Study text:
"EarthEnable installs affordable earthen floors in homes across Rwanda and Uganda, which helps mitigate health issues caused by dirt floors such as asthma, diarrhea, and malnutrition. The underside of EarthEnable’s flooring product had been suffering cracks and erosion at an unusually high rate, and they needed help diagnosing the cause."

In [9]:
test_text2 = "EarthEnable installs affordable earthen floors in homes across Rwanda and Uganda, which helps mitigate health issues caused by dirt floors such as asthma, diarrhea, and malnutrition. The underside of EarthEnable’s flooring product had been suffering cracks and erosion at an unusually high rate, and they needed help diagnosing the cause."
doc3 = nlp2(test_text2)
print("Entities in '%s'" % test_text2)
for ent in doc3.ents:
    print(ent.label_, ent.text)

Entities in 'EarthEnable installs affordable earthen floors in homes across Rwanda and Uganda, which helps mitigate health issues caused by dirt floors such as asthma, diarrhea, and malnutrition. The underside of EarthEnable’s flooring product had been suffering cracks and erosion at an unusually high rate, and they needed help diagnosing the cause.'
B-geo Rwanda
B-geo Uganda
B-geo .


In [18]:
test_text3 = "A marketing team at a leading advertising technology company had difficulties understanding how today’s marketers in Southeast Asia measure advertising effectiveness across different segments. The team also sought to differentiate their advertising offerings from key competitors."
doc4 = nlp2(test_text3)
print("Entities in '%s'" % test_text3)
for ent in doc4.ents:
    print(ent.label_, ent.text)

Entities in 'A marketing team at a leading advertising technology company had difficulties understanding how today’s marketers in Southeast Asia measure advertising effectiveness across different segments. The team also sought to differentiate their advertising offerings from key competitors.'
B-tim today
B-geo Southeast
I-geo Asia
B-geo .
B-geo .


In [19]:
test_text4 = "In November 2019, Tesla announced the release of the Cybertruck, the company’s all electric pickup truck and the automaker’s sixth vehicle since its founding. While initial research had been conducted, GLG clients still had questions regarding consumer sentiment about electric trucks and where the luxury brand’s truck fit into the marketplace before making an investment decision."
doc5 = nlp2(test_text4)
print("Entities in '%s'" % test_text4)
for ent in doc5.ents:
    print(ent.label_, ent.text)

Entities in 'In November 2019, Tesla announced the release of the Cybertruck, the company’s all electric pickup truck and the automaker’s sixth vehicle since its founding. While initial research had been conducted, GLG clients still had questions regarding consumer sentiment about electric trucks and where the luxury brand’s truck fit into the marketplace before making an investment decision.'
B-tim November
I-tim 2019
B-per Tesla
B-tim Cybertruck
B-org .
I-org While
B-org GLG
B-geo .
